# Exploración del Dataset: VACANTES

Importación de Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

# Configuración
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 50)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✅ Librerías importadas correctamente")

Carga del Dataset

In [ ]:
# Definir rutas de forma robusta
import os
from pathlib import Path

# Obtener la ubicación del notebook
notebook_dir = Path.cwd()
print(f" Directorio actual: {notebook_dir}")

# Construir ruta al directorio data/raw
if 'notebooks' in str(notebook_dir):
    DATA_RAW_PATH = Path('../../data/raw')
    DICT_PATH = Path('../../data/diccionarios')
else:
    DATA_RAW_PATH = Path(r'E:\MTPE\data\raw')
    DICT_PATH = Path(r'E:\MTPE\data\diccionarios')

print(f" Ruta DATA_RAW_PATH: {DATA_RAW_PATH.resolve()}")
print(f" Directorio existe: {DATA_RAW_PATH.exists()}")

# Cargar dataset
archivo_csv = DATA_RAW_PATH / 'Dataset_VACANTES.csv'
print(f" Archivo CSV: {archivo_csv.name}")
print(f" Archivo existe: {archivo_csv.exists()}")

if archivo_csv.exists():
    print("\n Cargando dataset (puede tardar unos segundos por el tamaño del archivo)...")
    df_vacantes = pd.read_csv(archivo_csv, encoding='utf-8-sig')
    print(f"\n Dataset cargado exitosamente")
    print(f" Dimensiones: {df_vacantes.shape[0]:,} filas x {df_vacantes.shape[1]} columnas")
else:
    print(f"\n ERROR: No se encuentra el archivo en: {archivo_csv.resolve()}")
    print(f"   Por favor, verifica la ruta del archivo.")

Vista Previa - Primeras Filas

In [ ]:
print(" Primeras 15 filas del dataset:")
display(df_vacantes.head(15))

Vista Previa - Últimas Filas

In [ ]:
print(" Últimas 10 filas del dataset:")
display(df_vacantes.tail(10))

Vista Previa - Muestra Aleatoria

In [ ]:
print(" Muestra aleatoria de 20 registros:")
display(df_vacantes.sample(min(20, len(df_vacantes)), random_state=42))

Información General del Dataset

In [ ]:
print(" Información del Dataset:")
print("=" * 80)
df_vacantes.info()

print("\n Columnas del Dataset:")
print("=" * 80)
for idx, col in enumerate(df_vacantes.columns, 1):
    print(f"{idx:2d}. {col} ({df_vacantes[col].dtype})")

Análisis de Valores Nulos

In [ ]:
print(" Análisis de Valores Nulos:")
print("=" * 80)

null_data = pd.DataFrame({
    'Columna': df_vacantes.columns,
    'Nulos': df_vacantes.isnull().sum(),
    '% Nulos': (df_vacantes.isnull().sum() / len(df_vacantes) * 100).round(2)
}).sort_values(by='Nulos', ascending=False)

display(null_data)

if null_data['Nulos'].sum() > 0:
    plt.figure(figsize=(14, 8))
    null_cols = null_data[null_data['Nulos'] > 0]
    plt.barh(null_cols['Columna'], null_cols['% Nulos'])
    plt.xlabel('Porcentaje de Valores Nulos (%)')
    plt.title('Valores Nulos por Columna - VACANTES')
    plt.tight_layout()
    plt.show()
else:
    print(" No hay valores nulos en el dataset")

Análisis de Duplicados

In [ ]:
print(" Análisis de Duplicados:")
print("=" * 80)

duplicados_totales = df_vacantes.duplicated().sum()
print(f"Registros duplicados (completos): {duplicados_totales:,} ({duplicados_totales/len(df_vacantes)*100:.2f}%)")

# Buscar columna de ID
id_cols = [col for col in df_vacantes.columns if 'ID' in col.upper()]
if id_cols:
    for id_col in id_cols:
        duplicados_id = df_vacantes[id_col].duplicated().sum()
        print(f"Duplicados en {id_col}: {duplicados_id:,} ({duplicados_id/len(df_vacantes)*100:.2f}%)")

        if duplicados_id > 0 and duplicados_id < 100:
            print(f"\nEjemplos de {id_col} duplicados:")
            ids_duplicados = df_vacantes[df_vacantes[id_col].duplicated(keep=False)][id_col].value_counts().head(10)
            display(ids_duplicados)

Estadísticas Descriptivas - Variables Numéricas

In [ ]:
print(" Estadísticas Descriptivas - Variables Numéricas:")
print("=" * 80)
display(df_vacantes.describe())

Estadísticas Descriptivas - Variables Categóricas

In [ ]:
print(" Estadísticas Descriptivas - Variables Categóricas:")
print("=" * 80)
display(df_vacantes.describe(include=['object']))

Análisis de Cardinalidad

In [ ]:
print(" Análisis de Cardinalidad:")
print("=" * 80)

# Calcular valores únicos para cada columna
valores_unicos = [df_vacantes[col].nunique() for col in df_vacantes.columns]
tipos_dato = [str(df_vacantes[col].dtype) for col in df_vacantes.columns]
porcentaje_cardinalidad = [(df_vacantes[col].nunique() / len(df_vacantes) * 100) for col in df_vacantes.columns]

cardinalidad = pd.DataFrame({
    'Columna': df_vacantes.columns,
    'Valores_Únicos': valores_unicos,
    'Tipo_Dato': tipos_dato,
    '% Cardinalidad': [round(p, 2) for p in porcentaje_cardinalidad]
}).sort_values(by='Valores_Únicos', ascending=False)

display(cardinalidad)

Distribución de Variables Categóricas - Top 20

In [ ]:
print("Distribución de Variables Categóricas (Top 20 valores):")
categorical_cols = df_vacantes.select_dtypes(include=['object']).columns

for col in categorical_cols:
    print(f"\n{'='*80}")
    print(f"Columna: {col}")
    print(f"{'='*80}")
    value_counts = df_vacantes[col].value_counts()
    print(f"Valores únicos: {df_vacantes[col].nunique():,}")
    print(f"\nTop 20 valores más frecuentes:")
    display(pd.DataFrame({
        'Valor': value_counts.head(20).index,
        'Frecuencia': value_counts.head(20).values,
        'Porcentaje': (value_counts.head(20).values / len(df_vacantes) * 100).round(2)
    }))

Análisis de Nombres de Avisos/Puestos

In [ ]:
print("Análisis de NOMBREAVISO (Títulos de Vacantes):")
print("=" * 80)

if 'NOMBREAVISO' in df_vacantes.columns:
    nombre_counts = df_vacantes['NOMBREAVISO'].value_counts()

    print(f"\nTotal de títulos de vacantes únicos: {df_vacantes['NOMBREAVISO'].nunique():,}")
    print(f"Valores nulos: {df_vacantes['NOMBREAVISO'].isna().sum():,}")
    print(f"\nTop 30 puestos más ofertados:")

    display(pd.DataFrame({
        'Título de Vacante': [str(x)[:80] for x in nombre_counts.head(30).index],
        'Cantidad': nombre_counts.head(30).values,
        'Porcentaje': (nombre_counts.head(30).values / len(df_vacantes) * 100).round(2)
    }))

    # Visualización
    plt.figure(figsize=(14, 10))
    top_nombres = nombre_counts.head(20)
    plt.barh(range(len(top_nombres)), top_nombres.values)
    plt.yticks(range(len(top_nombres)), [str(x)[:60] + '...' if len(str(x)) > 60 else str(x) for x in top_nombres.index])
    plt.xlabel('Cantidad de Vacantes')
    plt.title('Top 20 Títulos de Vacantes Más Ofertados')
    plt.tight_layout()
    plt.show()

    # Análisis de longitud de títulos
    df_vacantes['len_nombreaviso'] = df_vacantes['NOMBREAVISO'].fillna('').astype(str).str.len()

    print(f"\n Estadísticas de longitud de títulos:")
    print(f"   - Longitud mínima: {df_vacantes['len_nombreaviso'].min()}")
    print(f"   - Longitud máxima: {df_vacantes['len_nombreaviso'].max()}")
    print(f"   - Longitud promedio: {df_vacantes['len_nombreaviso'].mean():.1f} caracteres")
    print(f"   - Longitud mediana: {df_vacantes['len_nombreaviso'].median():.0f} caracteres")
else:
    print(" No se encontró columna NOMBREAVISO")

Análisis de Número de Vacantes y Postulantes

In [ ]:
print(" Análisis de VACANTES y POSTULANTES:")
print("=" * 80)

if 'VACANTES' in df_vacantes.columns:
    print(f"\n VACANTES (cantidad de puestos ofertados):")
    print(f"  - Mínimo: {df_vacantes['VACANTES'].min()}")
    print(f"  - Máximo: {df_vacantes['VACANTES'].max()}")
    print(f"  - Promedio: {df_vacantes['VACANTES'].mean():.2f}")
    print(f"  - Mediana: {df_vacantes['VACANTES'].median()}")
    print(f"  - Total vacantes ofertadas: {df_vacantes['VACANTES'].sum():,.0f}")

    # Distribución
    print(f"\n Distribución de número de vacantes:")
    value_counts = df_vacantes['VACANTES'].value_counts().sort_index().head(20)
    display(pd.DataFrame({
        'Num_Vacantes': value_counts.index,
        'Frecuencia': value_counts.values,
        'Porcentaje': (value_counts.values / len(df_vacantes) * 100).round(2)
    }))

if 'POSTULANTES' in df_vacantes.columns:
    print(f"\n POSTULANTES (cantidad de personas que aplicaron):")
    print(f"  - Mínimo: {df_vacantes['POSTULANTES'].min()}")
    print(f"  - Máximo: {df_vacantes['POSTULANTES'].max()}")
    print(f"  - Promedio: {df_vacantes['POSTULANTES'].mean():.2f}")
    print(f"  - Mediana: {df_vacantes['POSTULANTES'].median()}")
    print(f"  - Total postulantes: {df_vacantes['POSTULANTES'].sum():,.0f}")

    # Calcular ratio postulantes/vacantes
    if 'VACANTES' in df_vacantes.columns:
        df_vacantes['ratio_postulantes_vacantes'] = df_vacantes['POSTULANTES'] / df_vacantes['VACANTES'].replace(0, 1)
        print(f"\n Ratio Postulantes/Vacantes:")
        print(f"  - Promedio: {df_vacantes['ratio_postulantes_vacantes'].mean():.2f} postulantes por vacante")
        print(f"  - Mediana: {df_vacantes['ratio_postulantes_vacantes'].median():.2f} postulantes por vacante")
        print(f"  - Máximo: {df_vacantes['ratio_postulantes_vacantes'].max():.2f} postulantes por vacante")

# Visualizaciones
if 'VACANTES' in df_vacantes.columns and 'POSTULANTES' in df_vacantes.columns:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # Histograma de VACANTES
    axes[0, 0].hist(df_vacantes['VACANTES'].dropna(), bins=30, edgecolor='black')
    axes[0, 0].set_xlabel('Número de Vacantes')
    axes[0, 0].set_ylabel('Frecuencia')
    axes[0, 0].set_title('Distribución de Número de Vacantes por Aviso')

    # Boxplot de VACANTES
    axes[0, 1].boxplot(df_vacantes['VACANTES'].dropna())
    axes[0, 1].set_ylabel('Número de Vacantes')
    axes[0, 1].set_title('Boxplot de Vacantes')

    # Histograma de POSTULANTES
    axes[1, 0].hist(df_vacantes['POSTULANTES'].dropna(), bins=30, edgecolor='black')
    axes[1, 0].set_xlabel('Número de Postulantes')
    axes[1, 0].set_ylabel('Frecuencia')
    axes[1, 0].set_title('Distribución de Número de Postulantes por Aviso')

    # Scatter plot VACANTES vs POSTULANTES
    axes[1, 1].scatter(df_vacantes['VACANTES'], df_vacantes['POSTULANTES'], alpha=0.5)
    axes[1, 1].set_xlabel('Vacantes')
    axes[1, 1].set_ylabel('Postulantes')
    axes[1, 1].set_title('Relación Vacantes vs Postulantes')

    plt.tight_layout()
    plt.show()
else:
    print(" No se encontraron las columnas VACANTES y/o POSTULANTES")

Análisis Geográfico de Vacantes

In [ ]:
# Buscar columnas geográficas
geo_cols = [col for col in df_vacantes.columns if any(x in col.upper() for x in ['DEPARTAMENTO', 'PROVINCIA', 'DISTRITO', 'REGION', 'UBICACION'])]

if geo_cols:
    print("Análisis Geográfico de Vacantes:")
    print("=" * 80)

    for col_geo in geo_cols:
        print(f"\n{col_geo}:")
        geo_counts = df_vacantes[col_geo].value_counts()
        print(f"  - Valores únicos: {df_vacantes[col_geo].nunique():,}")
        print(f"  - Top 10:")
        display(pd.DataFrame({
            'Ubicación': geo_counts.head(10).index,
            'Cantidad': geo_counts.head(10).values,
            'Porcentaje': (geo_counts.head(10).values / len(df_vacantes) * 100).round(2)
        }))

    # Visualización del departamento si existe
    dept_cols = [col for col in geo_cols if 'DEPARTAMENTO' in col.upper()]
    if dept_cols:
        col_dept = dept_cols[0]
        dept_counts = df_vacantes[col_dept].value_counts().head(15)

        plt.figure(figsize=(14, 8))
        plt.barh(range(len(dept_counts)), dept_counts.values)
        plt.yticks(range(len(dept_counts)), dept_counts.index)
        plt.xlabel('Cantidad de Vacantes')
        plt.title('Top 15 Departamentos con más Vacantes')
        plt.tight_layout()
        plt.show()
else:
    print("No se encontraron columnas geográficas")

Análisis de Fechas de Publicación

In [ ]:
# Buscar columnas de fecha
fecha_cols = [col for col in df_vacantes.columns if 'FECHA' in col.upper() or 'DATE' in col.upper() or 'PUBLICACION' in col.upper()]

if fecha_cols:
    print(" Análisis de Fechas:")
    print("=" * 80)

    for col_fecha in fecha_cols:
        print(f"\n{col_fecha}:")
        print(f"  - Valores no nulos: {df_vacantes[col_fecha].notna().sum():,}")
        print(f"  - Valores nulos: {df_vacantes[col_fecha].isna().sum():,}")

        # Mostrar ejemplos
        df_temp = df_vacantes[col_fecha].dropna()
        if len(df_temp) > 0:
            print(f"  - Ejemplos: {df_temp.head(5).tolist()}")

            # Si es posible, mostrar rango de fechas
            print(f"  - Primero: {df_temp.min()}")
            print(f"  - Último: {df_temp.max()}")
else:
    print("No se encontraron columnas de fecha")

Análisis de Estado de Vacantes

In [ ]:
print(" Análisis de ACTIVO (Estado de Vacantes):")
print("=" * 80)

if 'ACTIVO' in df_vacantes.columns:
    estado_counts = df_vacantes['ACTIVO'].value_counts()

    print(f"\nEstados únicos: {df_vacantes['ACTIVO'].nunique()}")
    print(f"\nDistribución de estados:")

    display(pd.DataFrame({
        'Estado': estado_counts.index,
        'Cantidad': estado_counts.values,
        'Porcentaje': (estado_counts.values / len(df_vacantes) * 100).round(2)
    }))

    # Visualización
    plt.figure(figsize=(10, 6))
    plt.pie(estado_counts.values, labels=estado_counts.index, autopct='%1.1f%%', startangle=90)
    plt.title('Distribución de Estado de Vacantes')
    plt.axis('equal')
    plt.tight_layout()
    plt.show()

    # Análisis adicional
    if 'ACTIVO' in str(estado_counts.index[0]).upper():
        activos = estado_counts[estado_counts.index.str.upper() == 'ACTIVO'].sum() if len(estado_counts) > 0 else 0
        print(f"\n Vacantes ACTIVAS: {activos:,} ({activos/len(df_vacantes)*100:.1f}%)")
else:
    print(" No se encontró columna ACTIVO")

Análisis de Experiencia Requerida

In [ ]:
print("Análisis de Experiencia Requerida:")
print("=" * 80)

# Analizar SINEXPERIENCIA
if 'SINEXPERIENCIA' in df_vacantes.columns:
    print(f"\n SINEXPERIENCIA:")
    sinexp_counts = df_vacantes['SINEXPERIENCIA'].value_counts()

    display(pd.DataFrame({
        'Sin Experiencia': sinexp_counts.index,
        'Cantidad': sinexp_counts.values,
        'Porcentaje': (sinexp_counts.values / len(df_vacantes) * 100).round(2)
    }))

    # Visualización
    plt.figure(figsize=(8, 6))
    plt.pie(sinexp_counts.values, labels=sinexp_counts.index, autopct='%1.1f%%', startangle=90)
    plt.title('Vacantes que No Requieren Experiencia')
    plt.axis('equal')
    plt.tight_layout()
    plt.show()

# Analizar TIEMPOEXPERIENCIA y TIPOTIEMPOEXPERIENCIA
if 'TIEMPOEXPERIENCIA' in df_vacantes.columns:
    print(f"\n🔹 TIEMPOEXPERIENCIA (Tiempo de experiencia requerida):")

    # Convertir a numérico, forzando errores a NaN
    df_vacantes['TIEMPOEXPERIENCIA_num'] = pd.to_numeric(df_vacantes['TIEMPOEXPERIENCIA'], errors='coerce')

    # Filtrar valores válidos (entre 0 y 50 años es razonable)
    tiempo_validos = df_vacantes[(df_vacantes['TIEMPOEXPERIENCIA_num'] >= 0) &
                                   (df_vacantes['TIEMPOEXPERIENCIA_num'] <= 50)]

    errores_conversion = df_vacantes['TIEMPOEXPERIENCIA_num'].isna().sum()

    print(f"  ️  Valores con error de conversión: {errores_conversion:,} ({errores_conversion/len(df_vacantes)*100:.1f}%)")
    print(f"   Valores válidos para análisis: {len(tiempo_validos):,}")

    if len(tiempo_validos) > 0:
        print(f"\n   Estadísticas (solo valores válidos):")
        print(f"  - Mínimo: {tiempo_validos['TIEMPOEXPERIENCIA_num'].min()}")
        print(f"  - Máximo: {tiempo_validos['TIEMPOEXPERIENCIA_num'].max()}")
        print(f"  - Promedio: {tiempo_validos['TIEMPOEXPERIENCIA_num'].mean():.2f}")
        print(f"  - Mediana: {tiempo_validos['TIEMPOEXPERIENCIA_num'].median()}")

        # Distribución
        tiempo_counts = tiempo_validos['TIEMPOEXPERIENCIA_num'].value_counts().sort_index().head(20)
        display(pd.DataFrame({
            'Tiempo': tiempo_counts.index,
            'Frecuencia': tiempo_counts.values,
            'Porcentaje': (tiempo_counts.values / len(df_vacantes) * 100).round(2)
        }))

        # Histograma
        plt.figure(figsize=(12, 5))
        plt.hist(tiempo_validos['TIEMPOEXPERIENCIA_num'].dropna(), bins=30, edgecolor='black')
        plt.xlabel('Tiempo de Experiencia')
        plt.ylabel('Frecuencia')
        plt.title('Distribución de Tiempo de Experiencia Requerida (Valores Válidos)')
        plt.tight_layout()
        plt.show()
    else:
        print("   No hay valores válidos para analizar")

if 'TIPOTIEMPOEXPERIENCIA' in df_vacantes.columns:
    print(f"\n TIPOTIEMPOEXPERIENCIA (Unidad: años/meses):")
    tipo_counts = df_vacantes['TIPOTIEMPOEXPERIENCIA'].value_counts()

    display(pd.DataFrame({
        'Tipo': tipo_counts.index,
        'Cantidad': tipo_counts.values,
        'Porcentaje': (tipo_counts.values / len(df_vacantes) * 100).round(2)
    }))

# Análisis combinado
if all(col in df_vacantes.columns for col in ['SINEXPERIENCIA', 'TIEMPOEXPERIENCIA', 'TIPOTIEMPOEXPERIENCIA']):
    print(f"\n Análisis Combinado de Experiencia:")

    # Crear categorías con manejo de errores
    def categorizar_experiencia(row):
        try:
            if row['SINEXPERIENCIA'] == 'SI':
                return '0. Sin experiencia'

            # Intentar convertir tiempo a numérico
            tiempo = pd.to_numeric(row['TIEMPOEXPERIENCIA'], errors='coerce')

            if pd.isna(tiempo):
                return '5. Datos inválidos'

            tipo = str(row['TIPOTIEMPOEXPERIENCIA']).upper()

            # Si es en años
            if 'AÑO' in tipo or 'YEAR' in tipo:
                if tiempo <= 1:
                    return '1. Hasta 1 año'
                elif tiempo <= 2:
                    return '2. 1-2 años'
                elif tiempo <= 5:
                    return '3. 2-5 años'
                else:
                    return '4. Más de 5 años'
            # Si es en meses
            elif 'MES' in tipo or 'MONTH' in tipo:
                if tiempo <= 12:
                    return '1. Hasta 1 año'
                elif tiempo <= 24:
                    return '2. 1-2 años'
                elif tiempo <= 60:
                    return '3. 2-5 años'
                else:
                    return '4. Más de 5 años'
            else:
                return '1. Hasta 1 año'
        except:
            return '5. Datos inválidos'

    df_vacantes['categoria_experiencia'] = df_vacantes.apply(categorizar_experiencia, axis=1)
    cat_counts = df_vacantes['categoria_experiencia'].value_counts().sort_index()

    display(pd.DataFrame({
        'Categoría': cat_counts.index,
        'Cantidad': cat_counts.values,
        'Porcentaje': (cat_counts.values / len(df_vacantes) * 100).round(2)
    }))

    # Gráfico de barras
    plt.figure(figsize=(12, 6))
    plt.bar(range(len(cat_counts)), cat_counts.values)
    plt.xticks(range(len(cat_counts)), cat_counts.index, rotation=45, ha='right')
    plt.ylabel('Cantidad de Vacantes')
    plt.title('Distribución de Vacantes por Experiencia Requerida')
    plt.tight_layout()
    plt.show()

    print(f"\n Nota: Los datos inválidos requieren limpieza en la fase de ETL")
else:
    print(" No se encontraron todas las columnas de experiencia")

Análisis de Sectores y Clasificación ESCO

In [ ]:
print(" Análisis de SECTOR y Clasificación ESCO:")
print("=" * 80)

# Analizar SECTOR
if 'SECTOR' in df_vacantes.columns:
    print(f"\n SECTOR:")
    sector_counts = df_vacantes['SECTOR'].value_counts()

    print(f"  - Sectores únicos: {df_vacantes['SECTOR'].nunique():,}")
    print(f"  - Valores nulos: {df_vacantes['SECTOR'].isna().sum():,}")
    print(f"\n  Top 20 sectores:")

    display(pd.DataFrame({
        'Sector': sector_counts.head(20).index,
        'Cantidad': sector_counts.head(20).values,
        'Porcentaje': (sector_counts.head(20).values / len(df_vacantes) * 100).round(2)
    }))

    # Visualización
    plt.figure(figsize=(14, 8))
    top_sectores = sector_counts.head(15)
    plt.barh(range(len(top_sectores)), top_sectores.values)
    plt.yticks(range(len(top_sectores)), [str(x)[:50] + '...' if len(str(x)) > 50 else str(x) for x in top_sectores.index])
    plt.xlabel('Cantidad de Vacantes')
    plt.title('Top 15 Sectores con Más Vacantes')
    plt.tight_layout()
    plt.show()

# Analizar ESCO (Clasificación Europea de Ocupaciones)
if 'ESCO' in df_vacantes.columns:
    print(f"\nESCO (Clasificación Europea de Ocupaciones):")
    esco_counts = df_vacantes['ESCO'].value_counts()

    print(f"  - Clasificaciones ESCO únicas: {df_vacantes['ESCO'].nunique():,}")
    print(f"  - Valores nulos: {df_vacantes['ESCO'].isna().sum():,}")
    print(f"\n  Top 20 clasificaciones ESCO:")

    display(pd.DataFrame({
        'Clasificación ESCO': [str(x)[:80] for x in esco_counts.head(20).index],
        'Cantidad': esco_counts.head(20).values,
        'Porcentaje': (esco_counts.head(20).values / len(df_vacantes) * 100).round(2)
    }))

    # Visualización
    plt.figure(figsize=(14, 10))
    top_esco = esco_counts.head(15)
    plt.barh(range(len(top_esco)), top_esco.values)
    plt.yticks(range(len(top_esco)), [str(x)[:60] + '...' if len(str(x)) > 60 else str(x) for x in top_esco.index])
    plt.xlabel('Cantidad de Vacantes')
    plt.title('Top 15 Clasificaciones ESCO')
    plt.tight_layout()
    plt.show()

# Analizar ESPCD (Exclusivo para Personas Con Discapacidad)
if 'ESPCD' in df_vacantes.columns:
    print(f"\n ESPCD (Exclusivo para Personas con Discapacidad):")
    espcd_counts = df_vacantes['ESPCD'].value_counts()

    display(pd.DataFrame({
        'ESPCD': espcd_counts.index,
        'Cantidad': espcd_counts.values,
        'Porcentaje': (espcd_counts.values / len(df_vacantes) * 100).round(2)
    }))

    # Visualización
    plt.figure(figsize=(8, 6))
    plt.pie(espcd_counts.values, labels=espcd_counts.index, autopct='%1.1f%%', startangle=90)
    plt.title('Vacantes Exclusivas para Personas con Discapacidad')
    plt.axis('equal')
    plt.tight_layout()
    plt.show()

    exclusivas_pcd = df_vacantes[df_vacantes['ESPCD'] == 'SI'].shape[0]
    print(f"\nVacantes exclusivas para PCD: {exclusivas_pcd:,} ({exclusivas_pcd/len(df_vacantes)*100:.1f}%)")
else:
    print(" No se encontraron columnas de sector o clasificación")